# **_Librerías_**

In [1]:
# Análisis de datos y métodos númericos
import pandas as pd
import numpy as np

# Correlación Phi_K
import phik

# Tratamiento de variables categoricas
from sklearn import preprocessing

# Cálculo estadístico
from scipy.stats import chi2_contingency

# **_Tratamiento de Datos_**

In [2]:
# Lectura de datos
df = pd.read_csv('../Datos/incidentes-viales-c5.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1368503 entries, 0 to 1368502
Data columns (total 18 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   folio              1368503 non-null  object 
 1   fecha_creacion     1368503 non-null  object 
 2   hora_creacion      1368503 non-null  object 
 3   dia_semana         1368503 non-null  object 
 4   codigo_cierre      1368503 non-null  object 
 5   fecha_cierre       1368503 non-null  object 
 6   año_cierre         1368503 non-null  int64  
 7   mes_cierre         1368503 non-null  object 
 8   hora_cierre        1368503 non-null  object 
 9   delegacion_inicio  1368345 non-null  object 
 10  incidente_c4       1368503 non-null  object 
 11  latitud            1368060 non-null  float64
 12  longitud           1368068 non-null  float64
 13  clas_con_f_alarma  1368503 non-null  object 
 14  tipo_entrada       1368503 non-null  object 
 15  delegacion_cierre  1368363 non-n

In [4]:
df.isnull().sum()

folio                  0
fecha_creacion         0
hora_creacion          0
dia_semana             0
codigo_cierre          0
fecha_cierre           0
año_cierre             0
mes_cierre             0
hora_cierre            0
delegacion_inicio    158
incidente_c4           0
latitud              443
longitud             435
clas_con_f_alarma      0
tipo_entrada           0
delegacion_cierre    140
geopoint             446
mes                    0
dtype: int64

In [5]:
df.dropna(inplace=True)   # Borramos valores nulos (es una cantidad poco considerable)

In [6]:
df.duplicated().sum()

0

In [7]:
df.head()

,folio,fecha_creacion,hora_creacion,dia_semana,codigo_cierre,fecha_cierre,año_cierre,mes_cierre,hora_cierre,delegacion_inicio,incidente_c4,latitud,longitud,clas_con_f_alarma,tipo_entrada,delegacion_cierre,geopoint,mes
0,GA/160123/05714,23/01/2016,22:35:04,Sábado,(A) La unidad de atención a emergencias fue de...,24/01/2016,2016,Enero,00:21:08,VENUSTIANO CARRANZA,accidente-choque sin lesionados,19.422113,-99.084643,EMERGENCIA,BOTÓN DE AUXILIO,VENUSTIANO CARRANZA,"19.422113,-99.084643",1
1,AO/160123/05826,23/01/2016,22:50:49,Sábado,(A) La unidad de atención a emergencias fue de...,24/01/2016,2016,Enero,04:40:37,CUAJIMALPA,accidente-choque con lesionados,19.358320,-99.297641,URGENCIAS MEDICAS,BOTÓN DE AUXILIO,CUAJIMALPA,"19.35832,-99.297641",1
2,C4/160124/02187,24/01/2016,09:40:11,Domingo,(N) La unidad de atención a emergencias fue de...,24/01/2016,2016,Enero,11:13:03,TLALPAN,accidente-choque sin lesionados,19.217070,-99.219070,EMERGENCIA,LLAMADA DEL 066,TLALPAN,"19.21707,-99.21907008",1
3,C4/160124/05723,24/01/2016,22:40:57,Domingo,(N) La unidad de atención a emergencias fue de...,24/01/2016,2016,Enero,22:55:51,MAGDALENA CONTRERAS,accidente-choque sin lesionados,19.320580,-99.241010,EMERGENCIA,LLAMADA DEL 066,MAGDALENA CONTRERAS,"19.32057999,-99.24100992",1
4,C4/160124/01334,24/01/2016,04:25:15,Domingo,(A) La unidad de atención a emergencias fue de...,24/01/2016,2016,Enero,05:03:59,MIGUEL HIDALGO,accidente-choque sin lesionados,19.452900,-99.215870,EMERGENCIA,LLAMADA DEL 066,MIGUEL HIDALGO,"19.45289997,-99.21587004",1


In [8]:
# Tratamiento de variables

df['fecha_creacion'] = pd.to_datetime(df['fecha_creacion'])

df.rename(columns = {'mes':'mes_reporte'}, inplace=True)

df['año_reporte'] = df['fecha_creacion'].dt.year   # Creamos las columnas con la hora del
df['dia_numerico_reporte'] = df['fecha_creacion'].dt.day   # con el año, el día y la hora
df['hora_reporte'] = df.hora_creacion.astype(str).str.split(":").str[0]   # en que se reportó el incidente

In [9]:
df.año_reporte.unique()

array([2016, 2017, 2018, 2019, 2015, 2013, 2014, 2020])

In [10]:
df.dia_numerico_reporte.unique()

array([23, 24, 25, 26, 27, 28, 29, 30, 31,  2, 13, 20, 19, 21, 22,  3, 14,
       15, 16, 17, 18, 10,  5,  6,  4,  8,  9, 12,  7, 11,  1])

In [11]:
df.hora_reporte.unique()

array(['22', '09', '04', ..., '0.984409722', '0.987372685', '0.038634259'],
      dtype=object)

In [12]:
df.loc[df['hora_reporte'] == '0', 'hora_reporte'] = '00'

In [13]:
df[df.hora_reporte < '00']

,folio,fecha_creacion,hora_creacion,dia_semana,codigo_cierre,fecha_cierre,año_cierre,mes_cierre,hora_cierre,delegacion_inicio,...,latitud,longitud,clas_con_f_alarma,tipo_entrada,delegacion_cierre,geopoint,mes_reporte,año_reporte,dia_numerico_reporte,hora_reporte
865744,C5/200217/09463,2020-02-17,0.985266204,Lunes,(A) La unidad de atención a emergencias fue de...,18/02/2020,2020,Febrero,0.147476852,MIGUEL HIDALGO,...,19.45393,-99.19447,EMERGENCIA,LLAMADA DEL 911,MIGUEL HIDALGO,"19.45393002,-99.19446984",2,2020,17,0.985266204
865745,C5/200218/03696,2020-02-18,0.53494213,Martes,(A) La unidad de atención a emergencias fue de...,18/02/2020,2020,Febrero,0.592511574,AZCAPOTZALCO,...,19.49486,-99.19591,URGENCIAS MEDICAS,LLAMADA DEL 911,AZCAPOTZALCO,"19.49486004,-99.19590984",2,2020,18,0.53494213
865746,C5/200218/01827,2020-02-18,0.366539352,Martes,(A) La unidad de atención a emergencias fue de...,18/02/2020,2020,Febrero,0.486828704,XOCHIMILCO,...,19.25856,-99.08340,URGENCIAS MEDICAS,LLAMADA DEL 911,XOCHIMILCO,"19.25856,-99.08340012",2,2020,18,0.366539352
865747,C5/200218/03445,2020-02-18,0.51306713,Martes,(D) El incidente reportado se registró en dos ...,18/02/2020,2020,Febrero,0.520810185,MIGUEL HIDALGO,...,19.44439,-99.16717,EMERGENCIA,LLAMADA DEL 911,MIGUEL HIDALGO,"19.44439002,-99.16716996",2,2020,18,0.51306713
865748,C5/200218/03785,2020-02-18,0.541273148,Martes,(N) La unidad de atención a emergencias fue de...,18/02/2020,2020,Febrero,0.708449074,IZTAPALAPA,...,19.38405,-99.07583,EMERGENCIA,LLAMADA DEL 911,IZTAPALAPA,"19.38404997,-99.07583004",2,2020,18,0.541273148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1366020,C5/200906/07549,2020-06-09,0.822488426,Domingo,(A) La unidad de atención a emergencias fue de...,06/09/2020,2020,Septiembre,0.850972222,IZTACALCO,...,19.40757,-99.06515,EMERGENCIA,LLAMADA DEL 911,IZTACALCO,"19.40757003,-99.06514992",9,2020,9,0.822488426
1366021,C5/200906/07749,2020-06-09,0.840266204,Domingo,(A) La unidad de atención a emergencias fue de...,06/09/2020,2020,Septiembre,0.901643519,GUSTAVO A. MADERO,...,19.55014,-99.13305,URGENCIAS MEDICAS,LLAMADA DEL 911,GUSTAVO A. MADERO,"19.55014002,-99.13304988",9,2020,9,0.840266204
1366022,C5/200906/09239,2020-06-09,0.984409722,Domingo,(D) El incidente reportado se registró en dos ...,06/09/2020,2020,Septiembre,0.997372685,MAGDALENA CONTRERAS,...,19.30551,-99.25052,EMERGENCIA,LLAMADA DEL 911,MAGDALENA CONTRERAS,"19.30551003,-99.25052004",9,2020,9,0.984409722
1366023,C5/200906/09274,2020-06-09,0.987372685,Domingo,(D) El incidente reportado se registró en dos ...,06/09/2020,2020,Septiembre,0.989872685,IZTAPALAPA,...,19.33968,-99.10896,URGENCIAS MEDICAS,LLAMADA DEL 911,IZTAPALAPA,"19.33967997,-99.10896012",9,2020,9,0.987372685


In [14]:
df.drop(df[df.hora_reporte < '00'].index, inplace=True)   # Borramos los registros que acabamos de filtrar 

In [15]:
df['hora_reporte'] = df.hora_reporte.astype(int)

In [16]:
df.hora_reporte.unique()

array([22,  9,  4, 14, 23, 17, 16, 20, 10, 18,  1,  3, 13,  2,  7,  6,  5,
        8, 15, 19,  0, 11, 21, 12])

In [17]:
df.isnull().sum()

folio                   0
fecha_creacion          0
hora_creacion           0
dia_semana              0
codigo_cierre           0
fecha_cierre            0
año_cierre              0
mes_cierre              0
hora_cierre             0
delegacion_inicio       0
incidente_c4            0
latitud                 0
longitud                0
clas_con_f_alarma       0
tipo_entrada            0
delegacion_cierre       0
geopoint                0
mes_reporte             0
año_reporte             0
dia_numerico_reporte    0
hora_reporte            0
dtype: int64

In [18]:
df.duplicated().sum()

0

In [19]:
df['codigo_cierre'] = df.codigo_cierre.str.split(" ").str[0]   # Nos quedamos sólo con el código (borramos la descripción)

In [20]:
df1 = df.apply(lambda x: x.astype(str).str.lower() if(x.dtype == 'object') else x)   # Todas las columnas a minúscula

In [21]:
df1.drop(['hora_creacion','fecha_cierre','año_cierre','mes_cierre','hora_cierre',   # Borramos las columnas que
          'delegacion_cierre','geopoint'],axis='columns', inplace=True)             # no nos interesan

In [22]:
df1.to_pickle('../Datos/df1.pkl')

In [23]:
df1.head()

,folio,fecha_creacion,dia_semana,codigo_cierre,delegacion_inicio,incidente_c4,latitud,longitud,clas_con_f_alarma,tipo_entrada,mes_reporte,año_reporte,dia_numerico_reporte,hora_reporte
0,ga/160123/05714,2016-01-23,sábado,(a),venustiano carranza,accidente-choque sin lesionados,19.422113,-99.084643,emergencia,botón de auxilio,1,2016,23,22
1,ao/160123/05826,2016-01-23,sábado,(a),cuajimalpa,accidente-choque con lesionados,19.358320,-99.297641,urgencias medicas,botón de auxilio,1,2016,23,22
2,c4/160124/02187,2016-01-24,domingo,(n),tlalpan,accidente-choque sin lesionados,19.217070,-99.219070,emergencia,llamada del 066,1,2016,24,9
3,c4/160124/05723,2016-01-24,domingo,(n),magdalena contreras,accidente-choque sin lesionados,19.320580,-99.241010,emergencia,llamada del 066,1,2016,24,22
4,c4/160124/01334,2016-01-24,domingo,(a),miguel hidalgo,accidente-choque sin lesionados,19.452900,-99.215870,emergencia,llamada del 066,1,2016,24,4


In [24]:
df_verdaderos = df1[(df1['codigo_cierre'] == '(a)')]   # Nos quedamos sólo con los incidentes reales

In [25]:
df_verdaderos.drop(df_verdaderos[df_verdaderos.año_reporte == 2020].index, inplace=True)   # Borramos los incidentes del 2020 
                                                                                            # (no tenemos info sobre todos los meses aún)

df_verdaderos.drop(df_verdaderos[df_verdaderos.año_reporte == 2013].index, inplace=True)   # Lo mismo para 2013

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [26]:
df_verdaderos.to_pickle('../Datos/df_verdaderos.pkl')

In [27]:
df_verdaderos.head()

,folio,fecha_creacion,dia_semana,codigo_cierre,delegacion_inicio,incidente_c4,latitud,longitud,clas_con_f_alarma,tipo_entrada,mes_reporte,año_reporte,dia_numerico_reporte,hora_reporte
0,ga/160123/05714,2016-01-23,sábado,(a),venustiano carranza,accidente-choque sin lesionados,19.422113,-99.084643,emergencia,botón de auxilio,1,2016,23,22
1,ao/160123/05826,2016-01-23,sábado,(a),cuajimalpa,accidente-choque con lesionados,19.358320,-99.297641,urgencias medicas,botón de auxilio,1,2016,23,22
4,c4/160124/01334,2016-01-24,domingo,(a),miguel hidalgo,accidente-choque sin lesionados,19.452900,-99.215870,emergencia,llamada del 066,1,2016,24,4
6,ga/160123/05517,2016-01-23,sábado,(a),gustavo a. madero,accidente-choque con lesionados,19.467682,-99.095834,urgencias medicas,botón de auxilio,1,2016,23,22
7,c4/160124/03357,2016-01-24,domingo,(a),venustiano carranza,accidente-choque con lesionados,19.414310,-99.087160,urgencias medicas,radio,1,2016,24,14


## _Generando la **variable objetivo**_

### Variable objetivo: **_requiere_ambulancia_**. 1 = Si requiere, 0 = No requiere.

Dividiremos los incidentes en dos grupos. El primero serán aquellos incidentes en los que SI se necesitará ambulancia (aquellos con lesionados confirmados o por confirmar) y el segundo grupo correspondera a los incidentes que NO requieran atención medica. A saber:

### Incidentes que requieren ambulancia
* accidente-choque con lesionado
* accidente-otros
* lesionado-accidente automovilístico
* lesionado-atropellado
* accidente-volcadura
* accidente-persona atrapada / desbarrancada
* cadáver-accidente automovilístico
* cadáver-atropellado
* accidente-motociclista
* accidente-choque con prensados
* accidente-vehiculo atrapado
* accidente-vehiculo desbarrancado
* accidente-ciclista
* accidente-monopatín
* sismo-choque con lesionados
* accidente-vehículo atrapado-varado
* accidente-ferroviario
* sismo-persona atropellada
* detención ciudadana-atropellado
* detención ciudadana-accidente automovilístico
* sismo-choque con prensados

### Incidentes que **NO** requieren ambulancia
* accidente-choque sin lesionado
* sismo-choque sin lesionados

In [28]:
sin_ambulancia = ['accidente-choque sin lesionados','sismo-choque sin lesionados']
df_verdaderos['requiere_ambulancia'] = np.where(df_verdaderos['incidente_c4'].isin(sin_ambulancia),0,1)

<ipython-input-28-569fc41ef866>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_verdaderos['requiere_ambulancia'] = np.where(df_verdaderos['incidente_c4'].isin(sin_ambulancia),0,1)


## Correlaciones

### Phi_k

In [29]:
# Hacemos esta correlación sólo con el 0.25% de los registros de df_verdadero (con más registros mi computadora moría).
df_verdaderos_25 = (df_verdaderos.drop(['folio','fecha_creacion','codigo_cierre'], axis=1).sample(frac=0.0025, replace=True, random_state=1))

# Calculamos la correlación Phi_k entre pares de variables
phik = df_verdaderos_25.phik_matrix()

phik.to_pickle('../Datos/phik.pkl')
phik

interval columns not set, guessing: ['latitud', 'longitud', 'mes_reporte', 'año_reporte', 'dia_numerico_reporte', 'hora_reporte', 'requiere_ambulancia']


,dia_semana,delegacion_inicio,incidente_c4,latitud,longitud,clas_con_f_alarma,tipo_entrada,mes_reporte,año_reporte,dia_numerico_reporte,hora_reporte,requiere_ambulancia
dia_semana,1.000000,0.000000,0.114769,0.082972,0.106840,0.003211,0.072697,0.035334,0.044765,0.000000,0.126220,0.069910
delegacion_inicio,0.000000,1.000000,0.000000,0.854139,0.830299,0.000000,0.164458,0.000000,0.000000,0.000000,0.044982,0.000000
incidente_c4,0.114769,0.000000,1.000000,0.080678,0.188326,1.000000,0.123449,0.056914,0.205253,0.035370,0.204171,1.000000
latitud,0.082972,0.854139,0.080678,1.000000,0.440564,0.076794,0.139487,0.000000,0.000000,0.121221,0.000000,0.071463
longitud,0.106840,0.830299,0.188326,0.440564,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
clas_con_f_alarma,0.003211,0.000000,1.000000,0.076794,0.000000,1.000000,0.057066,0.000000,0.047835,0.008942,0.000000,0.994988
tipo_entrada,0.072697,0.164458,0.123449,0.139487,0.000000,0.057066,1.000000,0.000000,0.613993,0.000000,0.074530,0.068981
mes_reporte,0.035334,0.000000,0.056914,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.791985,0.065090,0.000000
año_reporte,0.044765,0.000000,0.205253,0.000000,0.000000,0.047835,0.613993,0.000000,1.000000,0.000000,0.000000,0.047857
dia_numerico_reporte,0.000000,0.000000,0.035370,0.121221,0.000000,0.008942,0.000000,0.791985,0.000000,1.000000,0.141839,0.032546


### Cramér

In [30]:
# Separamos las variables nominales
df_verdaderos_nominales = df_verdaderos[[i for i in df_verdaderos.columns if i not in ('folio','fecha_creacion','codigo_cierre','latitud','longitud',
                                                                                       'mes_reporte','año_reporte','dia_numerico_reporte','hora_reporte')]]
df_verdaderos_nominales.head()

,dia_semana,delegacion_inicio,incidente_c4,clas_con_f_alarma,tipo_entrada,requiere_ambulancia
0,sábado,venustiano carranza,accidente-choque sin lesionados,emergencia,botón de auxilio,0
1,sábado,cuajimalpa,accidente-choque con lesionados,urgencias medicas,botón de auxilio,1
4,domingo,miguel hidalgo,accidente-choque sin lesionados,emergencia,llamada del 066,0
6,sábado,gustavo a. madero,accidente-choque con lesionados,urgencias medicas,botón de auxilio,1
7,domingo,venustiano carranza,accidente-choque con lesionados,urgencias medicas,radio,1


In [31]:
# Tratamiento categórico
label = preprocessing.LabelEncoder()
df_nominales = pd.DataFrame() 

for i in df_verdaderos_nominales.columns:
    df_nominales[i] = label.fit_transform(df_verdaderos_nominales[i])

In [32]:
df_nominales.head()

,dia_semana,delegacion_inicio,incidente_c4,clas_con_f_alarma,tipo_entrada,requiere_ambulancia
0,5,14,2,1,0,0
1,5,4,0,3,0,1
2,0,10,2,1,3,0
3,5,6,0,3,0,1
4,0,14,0,3,5,1


In [33]:
# Consrucción de la funcion V Cramér
def cramers_V(var1,var2):
    crosstab = np.array(pd.crosstab(var1, var2, rownames=None, colnames=None))
    stat = chi2_contingency(crosstab)[0]
    obs = np.sum(crosstab)
    mini = min(crosstab.shape)-1
    return (stat/(obs*mini))

In [34]:
# Construcción de la matriz
rows = []

for var1 in df_nominales:
    col = []
    for var2 in df_nominales:
        cramers = cramers_V(df_nominales[var1], df_nominales[var2])
        col.append(round(cramers,2)) 
    rows.append(col)

cramers_results = np.array(rows)
cramer = pd.DataFrame(cramers_results, columns = df_nominales.columns, index =df_nominales.columns)

cramer.to_pickle('../Datos/cramer.pkl')
cramer

,dia_semana,delegacion_inicio,incidente_c4,clas_con_f_alarma,tipo_entrada,requiere_ambulancia
dia_semana,1.00,0.00,0.00,0.00,0.00,0.01
delegacion_inicio,0.00,1.00,0.00,0.00,0.00,0.01
incidente_c4,0.00,0.00,1.00,0.67,0.01,1.00
clas_con_f_alarma,0.00,0.00,0.67,1.00,0.00,0.87
tipo_entrada,0.00,0.00,0.01,0.00,1.00,0.01
requiere_ambulancia,0.01,0.01,1.00,0.87,0.01,1.00


## Shifts

In [35]:
no_shift = pd.DataFrame(df_verdaderos.groupby(['fecha_creacion','delegacion_inicio']).size())
shift_1 = pd.DataFrame(df_verdaderos.groupby(['fecha_creacion','delegacion_inicio']).size().shift(1)).dropna()
shift_2 = pd.DataFrame(df_verdaderos.groupby(['fecha_creacion','delegacion_inicio']).size().shift(2)).dropna()

In [36]:
print (len(no_shift))
print (len(shift_1))
print (len(shift_2))

34159
34158
34157


In [37]:
merge_shift = no_shift.merge(shift_1, left_index=True, right_index=True)

In [38]:
merge_shift = merge_shift.merge(shift_2, left_index=True, right_index=True)

In [39]:
merge_shift.columns = ['no_shift','shift_1','shift_2']

In [40]:
merge_shift.isnull().sum()

no_shift    0
shift_1     0
shift_2     0
dtype: int64

In [41]:
merge_shift

no_shift  shift_1  shift_2
fecha_creacion delegacion_inicio                              
2014-01-01     benito juarez               9      8.0     12.0
               coyoacan                    7      9.0      8.0
               cuauhtemoc                 13      7.0      9.0
               gustavo a. madero          39     13.0      7.0
               iztacalco                   5     39.0     13.0
...                                      ...      ...      ...
2019-12-31     milpa alta                  1     11.0      2.0
               tlahuac                     3      1.0     11.0
               tlalpan                    18      3.0      1.0
               venustiano carranza        19     18.0      3.0
               xochimilco                 13     19.0     18.0

[34157 rows x 3 columns]

In [42]:
tabla_modelo1 = df_verdaderos.drop(['folio','codigo_cierre','incidente_c4','latitud',
                                    'longitud','clas_con_f_alarma', 'tipo_entrada'], axis=1)

In [43]:
# Creando nuevas variables
fin_semana = ['sábado','domingo']

tabla_modelo1['fin_de_semana'] = np.where(tabla_modelo1['dia_semana'].isin(fin_semana),1,0) # 0 = No era fin de semana
                                                                                            # 1 = Era fin de semana

In [44]:
# Creamos y concatenamos variables dummies
delegacion_dummie = pd.get_dummies(tabla_modelo1.delegacion_inicio, drop_first=True)   # drop_firts para evitar
                                                                                       # multicolinealidad
tabla_modelo1 = pd.concat([tabla_modelo1,delegacion_dummie], axis= 1)

In [45]:
tabla_modelo1 = tabla_modelo1.drop(['dia_semana'], axis=1)

In [46]:
tabla_modelo1 = tabla_modelo1.set_index(['fecha_creacion','delegacion_inicio'])

In [47]:
tabla_modelo1 = tabla_modelo1.merge(merge_shift, left_index=True, right_index=True).reset_index()

In [48]:
tabla_modelo1.duplicated().sum()

167367

In [49]:
tabla_modelo1.drop_duplicates(inplace=True)

In [50]:
tabla_modelo1.duplicated().sum()

0

In [51]:
tabla_modelo1.isnull().sum()

fecha_creacion          0
delegacion_inicio       0
mes_reporte             0
año_reporte             0
dia_numerico_reporte    0
hora_reporte            0
requiere_ambulancia     0
fin_de_semana           0
azcapotzalco            0
benito juarez           0
coyoacan                0
cuajimalpa              0
cuauhtemoc              0
gustavo a. madero       0
iztacalco               0
iztapalapa              0
magdalena contreras     0
miguel hidalgo          0
milpa alta              0
tlahuac                 0
tlalpan                 0
venustiano carranza     0
xochimilco              0
no_shift                0
shift_1                 0
shift_2                 0
dtype: int64

In [52]:
tabla_modelo1.to_pickle('../Datos/tabla_modelo1.pkl')
tabla_modelo1.head()

,fecha_creacion,delegacion_inicio,mes_reporte,año_reporte,dia_numerico_reporte,hora_reporte,requiere_ambulancia,fin_de_semana,azcapotzalco,benito juarez,...,magdalena contreras,miguel hidalgo,milpa alta,tlahuac,tlalpan,venustiano carranza,xochimilco,no_shift,shift_1,shift_2
0,2014-01-01,benito juarez,1,2014,1,7,0,0,0,1,...,0,0,0,0,0,0,0,9,8.0,12.0
1,2014-01-01,benito juarez,1,2014,1,11,1,0,0,1,...,0,0,0,0,0,0,0,9,8.0,12.0
2,2014-01-01,benito juarez,1,2014,1,15,0,0,0,1,...,0,0,0,0,0,0,0,9,8.0,12.0
3,2014-01-01,benito juarez,1,2014,1,2,1,0,0,1,...,0,0,0,0,0,0,0,9,8.0,12.0
5,2014-01-01,benito juarez,1,2014,1,10,1,0,0,1,...,0,0,0,0,0,0,0,9,8.0,12.0
